In [1]:
import re
import string
import pandas as pd
from functools import reduce
from math import log

## Simple example of [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
1. Example of corpus
2. Preprocessing and Tokenizing
3. Calculating bag of words
4. TF
5. IDF
6. TF-IDF

In [6]:
#1
corpus = """
Simple example with Cats and Mouse
Another simple example with dogs and cats
Another simple example with mouse and cheese
""".split("\n")[1:-1]

corpus

['Simple example with Cats and Mouse',
 'Another simple example with dogs and cats',
 'Another simple example with mouse and cheese']

In [7]:
#2
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)

['simple', 'example', 'with', 'cats', 'and', 'mouse']
['another', 'simple', 'example', 'with', 'dogs', 'and', 'cats']
['another', 'simple', 'example', 'with', 'mouse', 'and', 'cheese']


In [8]:
#3
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'cats', 'example', 'another', 'simple', 'and', 'mouse', 'cheese', 'dogs', 'with'}


In [9]:
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

,and,another,cats,cheese,dogs,example,mouse,simple,with
0,1,0,1,0,0,1,1,1,1
1,1,1,1,0,1,1,0,1,1
2,1,1,0,1,0,1,1,1,1


## \#4 tf - term frequency
In the case of the term frequency $tf(t,d)$, the simplest choice is to use the raw count of a term in a string. 
$${\displaystyle \mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}} $$
where $n_t$ is the number of occurrences of the word $t$ in the string, and in the denominator - the total number of words in this string.

In [10]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [16]:
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

tf_A

{'cats': 0.16666666666666666,
 'example': 0.16666666666666666,
 'another': 0.0,
 'simple': 0.16666666666666666,
 'and': 0.16666666666666666,
 'mouse': 0.16666666666666666,
 'cheese': 0.0,
 'dogs': 0.0,
 'with': 0.16666666666666666}

## \#5 idf - inverse document frequency
idf is a measure of how much information the word provides
$$ \mathrm{idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|} $$
- $N$: total number of strings in the corpus ${\displaystyle N={|D|}}$
- ${\displaystyle |\{d\in D:t\in d\}|}$  : number of strings where the term ${\displaystyle t}$ appears (i.e., ${\displaystyle \mathrm {tf} (t,d)\neq 0})$. If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the denominator to ${\displaystyle 1+|\{d\in D:t\in d\}|}$.

In [17]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [18]:
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])
idf

{'cats': 0.4054651081081644,
 'example': 0.0,
 'another': 0.4054651081081644,
 'simple': 0.0,
 'and': 0.0,
 'mouse': 0.4054651081081644,
 'cheese': 1.0986122886681098,
 'dogs': 1.0986122886681098,
 'with': 0.0}

## \# 6 tf-idf
Then tf–idf is calculated as
$$ {\displaystyle \mathrm {tfidf} (t,d,D)=\mathrm {tf} (t,d)\cdot \mathrm {idf} (t,D)} $$

In [19]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [23]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [24]:
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])

,and,another,cats,cheese,dogs,example,mouse,simple,with
0,0.0,0.000000,0.067578,0.000000,0.000000,0.0,0.067578,0.0,0.0
1,0.0,0.057924,0.057924,0.000000,0.156945,0.0,0.000000,0.0,0.0
2,0.0,0.057924,0.000000,0.156945,0.000000,0.0,0.057924,0.0,0.0


# For clustering we must use tf-idf weights
the example above is just an example, in practice it is better to apply [TfidfVectorizer from sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Full text for clusterring

This corpus contain some strings about Google and some strings about TF-IDF from Wikipedia. Just for example

In [88]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
""".split("\n")[1:-1]
all_text


import os

path = '/home/akash/Documents/SLIIT/CDAP/Cropus/TestE/'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))


filenames = files
files = {}
 
for filename in filenames:
    with open(filename, "r") as file:
        if filename in files:
            continue
        files[filename] = file.read()
list=[]
for filename, text in files.items():
    list.append(text)
list

["On top of the world, after France defeated Croatia 4-2 to win its second-ever World Cup title, 19-year-old soccer star Kylain Mbappé now plans to donate his winnings to a children's charity.The French striker turned heads during the 2018 FIFA World Cup, netting four goals and getting named the best young player of the tournament.He's also an ambassador and huge supporter of Premiers de Cordée (PDC), an organization which offers free sports programs for disabled and hospitalized children.Ninon Barel, a spokesperson for the charity, confirmed about Mbappé's intentions to share his earnings with them but said he doesn't know the exact amount at this time.\n",
 "France won the World Cup for the second time by beating Croatia 4-2 in a tremendous final in Moscow.A breathtaking encounter featured the first World Cup final own goal, two hugely controversial refereeing decisions and a goalkeeping howler.France's victory meant Didier Deschamps, who captained them to glory in 1998, became just 

## Preprocessing and tokenizing
Firstly, we must bring every chars to lowercase and remove all punctuation, because it's not important for our task, but is very harmful for clustering algorithm. 
After that, we'll split strings to array of words.

In [89]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

Now, let's calculate tf-idf for this corpus

In [85]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(list)
tfidf

<9x523 sparse matrix of type '<class 'numpy.float64'>'
	with 1035 stored elements in Compressed Sparse Row format>

And train simple kmeans model with k = 2

In [91]:
kmeans = KMeans(n_clusters=2).fit(tfidf)
labels = kmeans.labels_
labels

array([1, 1, 1, 0, 0, 1, 0, 0, 0], dtype=int32)

Predictions

In [97]:
import os

path = '/home/akash/Documents/SLIIT/CDAP/Cropus/test2E/'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))


filenames = files
files = {}
 
for filename in filenames:
    with open(filename, "r") as file:
        if filename in files:
            continue
        files[filename] = file.read()
list2=[]
for filename, text in files.items():
    list2.append(text)
print(list2)


lines_for_predicting = ["මැලේසියාවේ ඉපෝ නුවරදී අවසන් වූ ආසියානු පළමු කොටසේ රග්බි තරගාවලියේදී ආසියානු ශ්‍රේණිගත කිරීමේ සිවුවැනි ස්ථානයේ පසු වූ ශ්‍රී ලංකාව ශ්‍රේණිගත කිරීමේ හත්වැනි ස්ථානයේ පසුවූ මැලේසියාව හමුවේ පරාජයට පත්වීමත් සමඟ ශ්‍රී ලංකාව ශ්‍රේණිගත කිරීමේ පහළට වැටීමේ අවදානමකට පත්ව තිබේ. ආසියානු තරගාවලියට පෙර ශ්‍රී ලංකාව ප්‍රසාද ලකුණු 49.79 ක් ලබා අන්තර්ජාතික ශ්‍රේණිගත කිරීම් අනුව ආසියාවේ සිවුවැනි ස්ථානයේද, කසකස්තානය ප්‍රසාද ලකුණු 48.14 ක් ලබා පස්වැනි ස්ථානයේද පසුවිය. ඒ වන විට මැලේසියාව ප්‍රසාද ලකුණු 43.53 ක් ලබා ආසියානු ශේණිගත කිරීම් අතරේ හත්වැනි ස්ථානයේ සිටියේය. ආසියානු ශ්‍රේණිගත කිරීමේ ලැයිස්තුව 2015 වසරේ නොවැම්බර් මස 23 වැනිදායින් පසුව යාවත්කාලීන කර නොමැත. ලෝක ශ්‍රේණිගත කිරීමේ මෙම මස 14 වැනිදා යාවත්කාලීන කිරීමෙන් පසු ශ්‍රී ලංකාව ප්‍රසාද ලකුණු 48.27 ක් ලබා 41 වැනි ස්ථානයේ ද, කසකස්තානය ප්‍රසාද ලකුණු 48.14 ක් ලබා 42 වැනි ස්ථානයේ ද, පසු වෙයි. ශ්‍රී ලංකාව ආසියානු තරගාවලියේදී ලෝක ශ්‍රේණිගත කිරීමේ 7 වැනි ස්ථානයේ සිටින මැලේසියාව හමුවේ පරාජයට පත්වීමෙන් කසකස්තානය ඉහළ පැමිණිමට ඉඩ තිබේ. ශ්‍රී ලංකා පිල එහිදී ලකුණු 22-9 ක් ලෙස මැලේසියාව හමුවේ පරාජයට පත්වූ අතර පිලිපීනය සහ එක්සත් අරාබි එමිර් රාජ්‍යය පරාජය කළේය. ආසියානු ශ්‍රේණිගත කිරීමේ හයවැනි ස්ථානයේ සිටින පිලිපීනය ලෝක ශ්‍රේණිගත කිරීමේ 57 වැනි ස්ථානයේ ද, 14 වැනි ස්ථානයේ පසු වන එක්සත් අරාබි එමිර් රාජ්‍යය ලෝක ශ්‍රේණිගත කිරීමේ 73 වැනි ස්ථානයේ ද, පසු වෙයි. ආසියානු ශ්‍රේණිගත කිරීමේ පළමු ස්ථානයේ සිට ජපානය ලෝක ශ්‍රේණිග කිරීමේ 11 වැනි ස්ථානයේ ද, දෙවැනි සහ තෙවැනි ස්ථානවල සිටින හොං කොං සහ දකුණු කොරියාව ලෝක ශ්‍රේණිගත කිරීමේ 26 සහ 28 වැනි ස්ථානවල ද පසු වෙති", "අවසන් මිනිත්තු දහය දක්වාම තම පිලට ප්‍රබල අභියෝගයක් එල්ල කරමින් ලකුණු දෙකකින් පමණක් පසුපසින් සිටි ශ්‍රී ලංකා ගුවන් හමුදා රග්බි කණ්ඩායමේ ක්‍රීඩකයන්ගේ නීතිවිරෝධී ක්‍රීඩා රටාව නිසා ලද දඬුවම් ලකුණු සංඛ්‍යාව සිය වාසියට හරවාගත් සී.ආර්.ඇන්ඞ් එෆ්.සී. ක්‍රීඩා සමාජය අන්තර් සමාජ පළමුපෙළ ඩයලොග් ලීග් රග්බි තරගාවලියේ අද (05 වැනිදා) පැවැති තරගයෙන් ලකුණු 21-9ක ජයක් අත්කර ගැනීමට සමත්විය. සී.ආර්. පිල ගෝලයක්, උත්සාහක දිනුමක් සහ දඬුවම් පහර 3 ක් ලබද්දී ගුවන් හමුදා පිලට දඬුවම් පහර 3 කින් පමණක් සෑහීමකට පත්වීමට සිදුවිය. තරගයේ පළමු භාගය නිමාවනවිටත් ලකුණු 6-3 කින් ඉදිරියෙන් පසුවූ ගුවන් හමුදා පිල අවසන් මිනිත්තු දහය එළැඹෙද්දී සී.ආර්.පිලට වඩා පසුපසින් සිටියේ ලකුණු 11-9 කින් පමණි. එහෙත් තරගය නිමාවීමට මිනිත්තු 7 කට පමණ පෙර ලද ‘පැනල්ටි ට්‍රයි ‘එකක් සහ අවසන් මිනිත්තුවේ ලද දඬුවම් පහරක් නිසා තරගය එහෙම පිටින්ම උඩුයටිකුරු වූයේ සී.ආර්.පිලට ජයග්‍රහණය අත්කර දෙමිනි. තරගයේ 8 වැනි මිනිත්තුවේදී දඬුවම් පහරක් සාර්ථක කරගත් චරිත් සෙනෙවිරත්න ලකුණු 3-0 කින් ගුවන් හමුදා පිල ඉදිරියට ගෙන ගියේය. ඊට මිනිත්තු දහයකට පසුව තම පිලට ලැබුණු දඬුවම් පහර සාර්ථක කරගත් තරින්ද රත්වත්ත ලකුණු සංඛ්‍යාව 3-3 කින් සම කරගත්තේය. අනතුරුව 36 වැනි මිනිත්තුවේ තවත් දඬුවම් පහරක් සාර්ථක කළ චරිත් සෙනෙවිරත්න ගුවන් හමුදා පිලට තරගයේ පළමු භාගයේ පෙරමුණ ගෙන දුන්නේ ලකුණු 6-3ක් ලෙසිනි. ගුවන් හමුදා පිලට වඩා ලකුණු 3ක් පසුපසින් සිටිමින් දෙවැනි භාගය ආරම්භ කළ සී.ආර්.පිලට නැවත පෙරමුණ ගෙනදුන් චරන චාමිකර 49 වැනි මිනිත්තුවේදී තරගයේ පළමු උත්සාහක දිනුම තැබුවේය. එය තරින්ද රත්වත්තට සාර්ථකව ප්‍රවර්ථනය කිරිමට නොහැකිවූවත් ලකුණු 8-6කින් පෙරමුණ ගැනීමට සී.ආර්.පිලට හැකිවිය", "Google", "Google Dev"]
kmeans.predict(tfidf_vectorizer.transform(list2))

["On top of the world, after France defeated Croatia 4-2 to win its second-ever World Cup title, 19-year-old soccer star Kylain Mbappé now plans to donate his winnings to a children's charity.The French striker turned heads during the 2018 FIFA World Cup, netting four goals and getting named the best young player of the tournament.He's also an ambassador and huge supporter of Premiers de Cordée (PDC), an organization which offers free sports programs for disabled and hospitalized children.Ninon Barel, a spokesperson for the charity, confirmed about Mbappé's intentions to share his earnings with them but said he doesn't know the exact amount at this time.\n", "France won the World Cup for the second time by beating Croatia 4-2 in a tremendous final in Moscow.A breathtaking encounter featured the first World Cup final own goal, two hugely controversial refereeing decisions and a goalkeeping howler.France's victory meant Didier Deschamps, who captained them to glory in 1998, became just t

array([1, 1, 0, 1, 0], dtype=int32)

In [93]:
from sklearn.metrics.cluster import completeness_score
completeness_score([1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1], [1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1])

0.7138429864106602

In [117]:
import csv

with open('persons11.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile)
    for ss in list2:
#         print(ss)
        filewriter.writerow([ss, 'cricket'])

In [2]:
isinstance("තරග තීරකවරුන්ගේ ආරක්ෂාව සම්බන්ධයෙන් දැඩි පියව", str)

True